In [40]:
from keras import layers
from keras import models
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os, shutil
from PIL import Image, ImageDraw, ImageFont, ImageColor
import random
import json

In [7]:
original_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project\\Training_set_6_9' 

base_dir='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project\\Training_base'
train_dir = os.path.join(base_dir, 'train')                                
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')                      
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')                                  
os.mkdir(test_dir)



In [59]:

#Loads labels and training data into two arrays, such that their indicies match
train_targets1=[]
with open("Training_set_6_9\\training_labels.json",'r') as dict_contents:
    train_targets_dict=json.load(dict_contents)

train_images1 = []
path = original_dataset_dir
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    #print(f[10:-4])
    train_images1.append(Image.open(os.path.join(path,f)))
    train_targets1.append(train_targets_dict[str(f[10:-4])])


[[13.5, 18.186533479473212],
 [[-7.0, 0.0], [3.5, 6.0621778264910695], [3.5, -6.0621778264910695]]]

In [63]:
#index_list=random.sample(range(len(train_images)),len(train_images))
index_list=[12, 77, 11, 32, 35, 28, 58, 70, 0, 46, 64, 74, 13, 1, 19, 2, 57, 39, 56, 30, 71, 49, 38, 7, 47, 75, 43, 25, 5, 44, 18, 50, 51, 17, 21, 27, 48, 29, 6, 10, 8, 34, 76, 9, 69, 60, 62, 15, 65, 53, 59, 16, 24, 4, 55, 22, 63, 78, 73, 67, 20, 42, 68, 52, 26, 79, 36, 33, 45, 41, 37, 72, 14, 40, 66, 3, 54, 31, 61, 23]
train_images=[]
train_targets=[]
for i in index_list:
    train_images.append(train_images1[i])
    train_labels.append(train_labels1[i])


'[12, 77, 11, 32, 35, 28, 58, 70, 0, 46, 64, 74, 13, 1, 19, 2, 57, 39, 56, 30, 71, 49, 38, 7, 47, 75, 43, 25, 5, 44, 18, 50, 51, 17, 21, 27, 48, 29, 6, 10, 8, 34, 76, 9, 69, 60, 62, 15, 65, 53, 59, 16, 24, 4, 55, 22, 63, 78, 73, 67, 20, 42, 68, 52, 26, 79, 36, 33, 45, 41, 37, 72, 14, 40, 66, 3, 54, 31, 61, 23]'

In [64]:
k = 4
num_val_samples = len(train_images) // k
num_epochs = 20
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_images = train_images[i * num_val_samples: (i + 1) * num_val_samples]    
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_images = np.concatenate(                                     
        [train_images[:i * num_val_samples],
         train_images[(i + 1) * num_val_samples:]],
         axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model()                                                    
    history= model.fit(partial_train_images, partial_train_targets,
                        validation_data=(val_images, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

processing fold # 0


TypeError: int() argument must be a string, a bytes-like object or a number, not 'PngImageFile'